
## Trial to combine middle ear and bruxism tagging to identify pure moments of middle ear activation

In [6]:
import os
PATH = os.getcwd() 
import sys
sys.path.append(PATH + '/../')
import mne
from tinnsleep.config import Config
from tinnsleep.create_reports import preprocess, reporting
from tinnsleep.data import CreateRaw, RawToEpochs_sliding, CleanAnnotations, AnnotateRaw_sliding
from tinnsleep.classification import AmplitudeThresholding
from tinnsleep.check_impedance import create_annotation_mne, Impedance_thresholding_sliding, check_RMS, fuse_with_classif_result
from tinnsleep.signal import rms
from tinnsleep.scoring import classif_to_burst, burst_to_episode, create_list_events
import warnings
import matplotlib.pyplot as plt
%matplotlib qt
import numpy as np
import pandas as pd
import pickle
import scipy
from tinnsleep.config import Config

print("Config loaded")


Config loaded


In [7]:
#List of MEMA files, hardcoded, to be modified
print(Config.bruxisme_files[44:46])
print("")
ME_files=[Config.bruxisme_files[5], Config.bruxisme_files[9], 
          Config.bruxisme_files[14], Config.bruxisme_files[22]]
ME_files.extend(Config.bruxisme_files[28:44])
ME_files.extend(Config.bruxisme_files[46:])

print(ME_files)
print(len(ME_files))

['E:/Acou_sommeil/EDF_V2_PAUL\\robin_nuit_23_sept.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\robin_nuit_son_24_sept.edf']

['E:/Acou_sommeil/EDF_V2_PAUL\\1DA15_nuit_son.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1GB19_nuit_hab.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1MF19_nuit_hab.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1RA17_nuit_hab.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\HZB_nuit_1.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\HZB_nuit_2.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\HZB_nuit_3.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHMIDTLIN_nuit_1_dec_OD__0to0.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHMIDTLIN_nuit_3_dec_OD__4to0to2.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHMIDTLIN_nuit_4_dec_OD__3to3.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHMIDTLIN_nuit_5_dec_OD__0to1.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHM_nuit_1.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHM_nuit_2.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHM_nuit_3.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\Schmidtlin_nuit_2_dec_3to0to4.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\Unger_2.edf', 'E:/Acou_somm

## Setting down parameters for bruxism detection

In [8]:
#Setting parameters
os.chdir("C:/Users/Zeta/Documents/acou_sommeil_HD_ENS/Tinnitus-n-Sleep/Notebooks")
THR_classif=[[0,2]]
sfreq = 250
window_length = 0.25                    # in seconds
duration = int(window_length * sfreq)   # in samples
interval = duration                     # no overlapping
n_adaptive = 480 # number of epochs for adaptative baseline

#Importing personnalized parameters for dataset
df = pd.read_pickle("data/valid_chans_THR_imp.pk")
dico_chans= df.to_dict("list")

delim =3
print("parameters set")


parameters set


## Bruxism + MEMA processing for pure MEMA visualisation

In [11]:
filenames = ME_files[16:17]  # load file from config
#filenames=['E:/Acou_sommeil/EDF_V2_PAUL\\sophie_mema.edf']

#Output of the processing, stock pure MEMA analysis output
ME_reports={}
results={}

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    
    print("Files processed : ")
    
    #Loop on all the patient files
    for filename in filenames:
        
        #-----------------Preparation for bruxism  processing ---------------------------------------
        #opens the raw file
        raw = mne.io.read_raw_edf(filename, preload=False, verbose=False)  # prepare loading
        print(filename.split("\\")[-1])
        #Get channels indexes
        ind_picks_chan= dico_chans[filename.split("\\")[-1]][0]
        ind_picks_imp= dico_chans[filename.split("\\")[-1]][1]
        #Get THR_imp value for filename
        THR_imp = dico_chans[filename.split("\\")[-1]][2]
        #Get channel names from indexes
        if len(ind_picks_chan)>0: #ignore file if no channel is good
            picks_chan=[]
            for elm in ind_picks_chan:
                picks_chan.append(raw.info["ch_names"][elm])
            picks_imp=[]
            for elm in ind_picks_imp:
                picks_imp.append(raw.info["ch_names"][elm])
            print(picks_chan)
            #Setting parameters for is_good
            params = dict(ch_names=picks_chan,
                  rejection_thresholds=dict(emg=5e-04),  # two order of magnitude higher q0.01
                  flat_thresholds=dict(emg=1e-09),  # one order of magnitude lower median
                  channel_type_idx=dict(emg=[ i for i in range(len(picks_chan))]),
                  full_report=True
                  )
            #Epoching parameters
            window_length = 0.25                    # in seconds
            duration = int(window_length * sfreq)   # in samples
            interval = duration                     # no overlapping
            
            
            #-----------------Bruxism  processing ---------------------------------------
            # Get the preprocessing steps done
            epochs, valid_labels, log = preprocess(raw, picks_chan, picks_imp, duration, interval, 
                                                   params, THR_imp=THR_imp, get_log=True)
            if np.sum(valid_labels)>0 : #If at least one epoch is good create report
                results[filename] = reporting(epochs, valid_labels, THR_classif, time_interval=window_length, delim=delim, n_adaptive = n_adaptive, log=log)
                #List of labels for MEMA processing crossing
                bruxism = results[filename]["labels"][0]
                artefacts = np.invert(valid_labels)
                
                #------------grouping bruxism episodes and bursts together------------------------------------
                burst_list = classif_to_burst(bruxism, time_interval=0.25)
                li_ep = burst_to_episode(burst_list, delim=3)
                event_list = create_list_events(li_ep, 0.25, len(bruxism) * 0.25)
                
                #All episodes as tonic
                for i in range(len(event_list)):
                    if event_list[i]!=0:
                        event_list[i]=True
                    else:
                        event_list[i] = False

                bruxism_c = np.any(np.c_[bruxism, event_list], axis=-1) #rassemblement des bursts et des episodes de bruxismes

            
                print("Bruxism reporting done")

                #-----------------MEMA processing preparation ---------------------------------------
                picks_chan = ['Airflow']           # middle ear electrode
                raw  = CreateRaw(raw[picks_chan][0], raw.info["sfreq"], picks_chan, ch_types=['emg']) # pick channels and load
                ch_names = raw.info["ch_names"]
                print("Data filtered")

                #epoching
                sfreq = raw.info["sfreq"]
                window_length = 1                    # in seconds
                duration = int(window_length * sfreq)   # in samples
                interval = duration                     # no overlapping
                epochs = RawToEpochs_sliding(raw, duration=duration, interval=interval)
                print(f"Epochs done, shape {epochs.shape}")

                #-----------------MEMA processing Foward-backward ---------------------------------------
                #Foward
                # compute the sum of power over electrodes and samples in each window
                pipeline = AmplitudeThresholding(abs_threshold=0., rel_threshold=4, n_adaptive=60)
                X        = rms(epochs) # take only valid labels
                labels_f   = pipeline.fit_predict(X)


                #Backward
                #Reversing epochs array
                epochs = epochs[::-1]
                 # compute the sum of power over electrodes and samples in each window
                pipeline = AmplitudeThresholding(abs_threshold=0., rel_threshold=4, n_adaptive=60)
                X        = rms(epochs) # take only valid labels
                labels   = pipeline.fit_predict(X)
                #Reversing labels
                labels_b = labels[::-1]
                
                
                #-----------------MEMA foward-backward merge and epochs length adaptation---------------------------------------
                # Logical OR -- merged backward and foward
                labels_fb = np.any(np.c_[labels_f, labels_b], axis=-1)
                
                #adaptation of labels_fb from 1s epochs to 0,25s epochs
                labels_fb_shep=[]  
                for elm in labels_fb:
                    for i in range(4):
                        labels_fb_shep.append(elm)
                 
                #-----------------Pure MEMA bursts conversion to episodes ----------------------------------------
                OM_burst = classif_to_burst(labels_fb_shep, time_interval=0.25)
                OM_ep= burst_to_episode(OM_burst, delim=2, min_burst_joining=0)
                li_OM = create_list_events(OM_ep, 0.25, 0.25* len(bruxism))
                
                burst_list = classif_to_burst(bruxism, time_interval=0.25)
                
                
                #---------------Classifying MEMA events according to bruxism----------------------
                comb_ep=[]
                pure_ep=[]
                compt_arti=0
                #merge MEMA and compare with bruxism and artefacts
                for elm in OM_ep:
                    if np.sum(artefacts[int(elm.beg/0.25):int(elm.end/0.25)]) == 0:
                        if np.sum(bruxism_c[int(elm.beg/0.25):int(elm.end/0.25)])>0 :
                            comb_ep.append(elm)
                        else:
                            pure_ep.append(elm)


                    else:
                        compt_arti+=1
                
                #------------------
                #Pure episodes creation
                li_OM_p = create_list_events(pure_ep, 0.25, 0.25* len(bruxism))
                #Combined episode creation
                li_OM_c = create_list_events(comb_ep, 0.25, 0.25* len(bruxism))



                #All episodes as tonic
                for i in range(len(li_OM_p)):
                    if li_OM_p[i]!=0:
                        li_OM_p[i]=2
                    else:
                        li_OM_p[i] = False

                #All episodes as tonic
                for i in range(len(li_OM_c)):
                    if li_OM_c[i]!=0:
                        li_OM_c[i]=3
                    else:
                        li_OM_c[i] = False



                print("Total number of middle ear episodes")
                print(len(OM_ep))
                print("Number of pure episodes")
                print(len(pure_ep))
                print("Number of combined episodes")
                print(len(comb_ep))
                print("Number of episodes rejected due to artifacts:")
                print(compt_arti)



                #-----------------Pure MEMA episodes visualisation and comparison with ATM activity -----------------------------------
                #Preparing raw for visualisation
                picks_chan = ['Airflow', '1', '2']           # subset of EMG electrodes
                raw  = mne.io.read_raw_edf(filename, preload=False, verbose=False)  # prepare loading
                raw  = CreateRaw(raw[picks_chan][0], raw.info["sfreq"], picks_chan, ch_types='emg')        # pick channels and load
                raw  = raw.load_data()
                dat=raw.get_data()
                dat[1]=[dat[1][i]*1/(0.0005) for i in range(len(dat[1]))]
                dat[2]=[dat[2][i]*1/(0.0005) for i in range(len(dat[2]))]
                raw  = CreateRaw(dat, raw.info["sfreq"], picks_chan, ch_types='emg') 

                raw  = raw.filter(20., 99., n_jobs=4, 
                                  fir_design='firwin', filter_length='auto', phase='zero-double',
                                  picks=['1', '2'])

                #Annotating the raw
                raw = CleanAnnotations(raw)
                dict_annotations = {1: "bruxism", 2:"pure", 3:"comb", 4:"tot"}
                raw = AnnotateRaw_sliding(raw, li_OM_p, 
                                dict_annotations=dict_annotations, duration=50, interval=50)
                raw = AnnotateRaw_sliding(raw, li_OM_c, 
                                dict_annotations=dict_annotations, duration=50, interval=50)

                print("Raw annotated")
                raw.plot(scalings = "auto")
                plt.title(filename)



Files processed : 
jon_mema.edf
['1', '2']
Bruxism reporting done
Data filtered
Epochs done, shape (33680, 1, 250)
Total number of middle ear episodes
239
Number of pure episodes
197
Number of combined episodes
35
Number of episodes rejected due to artifacts:
7
Raw annotated


## Small function to check if all channel selections are good

In [5]:
import warnings

labels_subj={}
EDF_list = Config.bruxisme_files

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    
    results={}
    print("Files processed : ")
    for filename in EDF_list:
        #opens the raw file
        raw = mne.io.read_raw_edf(filename, preload=False, verbose=False)  # prepare loading
        print(filename.split("\\")[-1])
        #Get channels indexes
        ind_picks_chan= dico_chans[filename.split("\\")[-1]][0]
        ind_picks_imp= dico_chans[filename.split("\\")[-1]][1]
        #Get THR_imp value for filename
        THR_imp = dico_chans[filename.split("\\")[-1]][2]
        #print(raw.info["ch_names"])
        #Get channel names from indexes
        if len(ind_picks_chan)>0: #ignore file if no channel is good
            picks_chan=[]
            for elm in ind_picks_chan:
                picks_chan.append(raw.info["ch_names"][elm])
            picks_imp=[]
            for elm in ind_picks_imp:
                picks_imp.append(raw.info["ch_names"][elm])
            print(picks_chan)
            print(picks_imp)
            print("")

Files processed : 
1BA07_nuit_hab.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1BA07_nuit_son.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1CC05_nuit_hab.edf
['1', '2']
['1 Impedance', '2 Impedance']

1CC05_nuit_son.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1DA15_nuit_hab.edf
['1']
['1 Imp?dance']

1DA15_nuit_son.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1DL12_nuit_hab.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1DL12_nuit_son.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1GB18_nuit_son.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1GB19_nuit_hab.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1GF14_nuit_hab.edf
1GF14_nuit_son.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1MA16_nuit_hab.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1MA16_nuit_son.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1MF19_nuit_hab.edf
['1', '2']
['1 Imp?dance', '2 Imp?dance']

1MF19_nuit_son.edf
['1', '2']
['1 Impedance', '2 Impedance']

1MN09_nuit_hab.edf
['1', '2']
['1 Imp?dance', '2 Imp